# Kurs 9

## Zufallszahlen

Für viele Anwendungen braucht man Zufallszahlen. Zwar haben wir in Julia nur Pseudozufallszahlen (die Zahlen werden nach einer deterministischen Methode berechnet), die sich zudem irgendwann wiederholen (das nennt man Periode). Allerdings verhalten sich die Zahlen im besten Fall sehr ähnlich zu tatsächlich zufällig (aus einer Verteilung) gezogenen Zahlen und die Periode ist oft so groß, dass sie in der Praxis keine Rolle spielt (für den [Mersenne-Twister](https://de.wikipedia.org/wiki/Mersenne-Twister): `4.3*10^6001`).

In [ ]:
using Pkg
Pkg.add("Random")

In [ ]:
using Random

Zentral sind die ab Werk verfügbaren Sampler der uniformen Verteilung und Normalverteilung:

In [ ]:
rand() # uniform verteilter Wert

In [ ]:
randn() # normalverteilter Wert

Um zu zeigen, dass die Funktionen funktionieren wie gedacht, ist hier ein Histogramm mit Kerndichteschätzer (wenn einem das nichts sagt – einfach ignorieren).

In [ ]:
Pkg.add("KernelDensity")

In [ ]:
using Plots
using KernelDensity

data = randn(100000)
density = kde(data)
histogram(
    data,
    normalize = true
)
plot!(density.x, density.density, linewidth=2, label="KDE")

Um reproduzierbare Zufallszahlen zu generieren, setzen wir einen sogenannten *seed*. Das ist extrem nützlich, wenn wir zum Beispiel Simulationen in einer Studie überprüfbar machen wollen.

In [ ]:
println(rand(2))
println(rand(2))

Random.seed!(1) # setze den seed auf 1
println(rand(2))
println(rand(2))

Random.seed!(1) # setze den seed auf 1
println(rand(2))

### Distributions.jl

Das Paket Distributions.jl stellt uns viele weitere Verteilungen bereit.

In [ ]:
Pkg.add("Distributions")
using Distributions

In [ ]:
rand(Cauchy())

## Performance

Julia ist per se ziemlich schnell – das ist ja gerade einer der Gründe, warum wir diese Sprache nutzen. Allerdings gibt es für ein bestimmtes Problem oft viele Wege nach Rom, wobei manche effizienter als andere sind. In manchen Fällen ist es sogar so, dass die Suche nach Optimierungen ein Fass ohne Boden ist.

Gerade deshalb sollte man sich aber nicht immer den Kopf über jedes Detail zerbrechen, das möglicherweise performancerelevant ist. Donald E. Knuth (legendärer Programmierer und unter anderem Erfinder von TeX) sagte dazu mal: „Premature optimization is the root of all evil“. In anderen Worten: Meistens sollte man eher versuchen schönen generischen Code zu schreiben und sich hinterher um Details kümmern, als sich umgekehrt in diesen zu verrennen – was nicht nur Zeit kostet, sondern im schlimmsten Fall zu Spaghetticode führt (siehe auch [hier](https://wiki.c2.com/?PrematureOptimization)).

Sinnvoll ist dagegen zum Beispiel ein sogenannter Profiler (etwa `@profview`), der uns Fingerzeige liefert, wo möglicherweise Speed flöten geht.

### Type instabilities

Julia ist deshalb schnell, weil es für unterschiedliche Typen als Inputs auch tatsächlich verschiedene Funktionen (oder Codeabschnitte) in Assembler kompiliert. Man muss also nicht innerhalb der Funktion sozusagen wieder abchecken: "Was wäre, wenn hier jetzt diese Operation mit Typ X wäre?". Stattdessen wird die Funktion von vornherein spezialisiert. Umgekehrt müssen wir diesen Prozess nicht händisch wie in C/C++ machen, stattdessen wird das für uns von Julia übernommen.

Es gibt allerdings Fälle, in denen ist Julia sich nicht sicher, ob sich der Typ innerhalb eines Codeabschnitts ändern kann und geht deshalb auf Nummer sicher. Soll heißen: Dieser Codeabschnitt ist dann weniger stark spezialisert und operiert üblicherweise auf Typen der Art ```Union{Typ_1, Typ_2}```. Das ist auch einer der Gründe, warum man *globale Variablen* vermeiden sollte (wenngleich sich hier nicht nur der Typ, sondern zusätzlich auch der Wert unvorhergesehen ändern kann). Für die, die es genauer wissen wollen, [hier](https://discourse.julialang.org/t/why-type-instability/4013/19) ein kleiner Thread zur Frage, warum es überhaupt type instabilities gibt.

#### Realitätsnahes Beispiel 1

In [ ]:
Pkg.add("BenchmarkTools")

In [ ]:
using BenchmarkTools

```@time```, printed die benötigte Zeit und gibt das Ergebnis zurück. Aber Achtung: Beim ersten Ausführen muss der Code erst kompiliert, sodass wir beim ersten Mal immer langsamer sind!. Deshalb benutzen wir das Makro ```@btime```, das mehrere Durchläufe macht und die Zeit zum Kompilieren ignoriert.

In [ ]:
aa = Any[1:1000;];
ai = [1:1000;];

@btime sum($aa)
@btime sum($ai)

In [ ]:
@code_warntype sum(aa)

In [ ]:
@code_warntype sum(ai)

#### Realitätsnahes Beispiel 2

In [ ]:
struct RealPoint
    x::Real
    y::Real
end

Hier haben wir den abstrakten Typ `Real` als Typ unserer Felder verwendet. Das funktioniert zwar, aber ist nicht besonders toll. Denn `Real` ist kein konkreter Typ: Der Julia-Compiler kann keine Annahmen über das Datenlayout von RealPoint treffen, und die Methodenauswahl muss zur Laufzeit und nicht zur Kompilierzeit erfolgen. Der richtige Weg, dies zu tun, wäre wie folgt:

In [ ]:
struct Point{T<:Real}
    x::T
    y::T
end

### Row vs. column major

#### Konzept

In den meisten Programmiersprachen sind Matrizen bzw. Arrays technisch nichts anderes als eindimensionale Arrays, also quasi Vektoren (Achtung: Wir reden hier nicht von echten Typen der Art ```Vector```). Das bedeutet: Elemente werden zeilen- oder spaltenweise (row-/column major) sukzessive in einer langen Liste gespeichert.

![Alt text](../graphics/rowcolumnarrays.png)

Es kommt ein bisschen auf Hardware bzw. konkrete Architektur an, aber generell gilt: Man möchte tendenziell auf (physisch) nahe beieinanderliegende Elemente zugreifen, weil der Zugriff dann schneller erfolgt. In anderen Worten: Im Algorithmus sollten häufige Sprünge möglichst vermieden werden!

Ein Beispiel dafür wäre die Implementierung einer Matrixmultiplikation.

#### Beispiel: Matrixmultiplikation

In [ ]:
Pkg.add("BenchmarkTools")

In [ ]:
using BenchmarkTools

m = 1000; n = 1000; k = 1000
X = rand(m, k)
Y = rand(k, n)
Z = zeros(m, n)

function naive_matmul!(A, B, C)
    C .= 0
    for i in 1:size(A)[1]
        for j in 1:size(B)[2]
            for k in 1:size(A)[2]
                @inbounds C[i, j] += A[i, k] * B[k, j]
            end
        end
    end
end

@btime naive_matmul!(X, Y, Z)
isapprox(Z, X * Y, atol = 1e-10)

In [ ]:
# lediglich Reihenfolge j, k, i ist anders
function smart_matmul!(A, B, C)
    C .= 0
    # @simd bringt hier scheinbar nichts
    @simd for j in 1:size(B)[2]
        for k in 1:size(A)[2]
            for i in 1:size(A)[1]
                @inbounds C[i, j] += A[i, k] * B[k, j]
            end
        end
    end
end

@btime smart_matmul!(X, Y, Z)
isapprox(Z, X * Y, atol = 1e-10)

Hieran kann man auch gut erkennen, dass man in der Regel keine neuen Objekte erzeugen, sondern auf bereits existierende operieren möchte. Konkret: Die Matrix C wird nicht neu definiert und mit ```return``` zurückgegeben. Denn wäre dies der Fall, so müsste bei jedem Aufruf der Matrixmultiplikation neuer Speicher angelegt werden. Und wenn wir das öfters tun, kostet es Zeit.

### Ausblick

Viele Probleme, die aus mathematischer Sicht trivial sind, sind in der Implementierung alles andere als das. Wie man hier etwa sehen kann, sind wir immer noch mindestens Faktor 10 von einer wirklich performanten Version entfernt:

In [ ]:
@btime Z = X * Y

Ein großer Teil dieser Optimierung AVX.

In [ ]:
Pkg.add("LoopVectorization")

In [ ]:
# exportiert @turbo für AVX
using LoopVectorization

@inline function avx_matmul!(A, B, C)
    C .= 0
    @turbo for j in 1:size(B)[2]
        for k in 1:size(A)[2]
            for i in 1:size(A)[1]
                @inbounds C[i, j] += A[i, k] * B[k, j]
            end
        end
    end
end

@btime avx_matmul!(X, Y, Z)

Der Rest ist hauptsächlich 
- memory modelling
- Cache/blocking inlining
- packing, padding
und stark von der Hardware abhängig.

Natürlich ist das extrem abhängig von Matrixgröße und LoopVectorization besser, wenn caching keine große Rolle spielt. Siehe auch [hier](https://discourse.julialang.org/t/ann-loopvectorization/32843) und [Docs](https://juliasimd.github.io/LoopVectorization.jl/latest/examples/matrix_multiplication/). 

Hier beispielsweise ein interessanter [Thread]((https://discourse.julialang.org/t/julia-matrix-multiplication-performance/55175/18)), wie im Paket Octavian.jl Matrixmultiplikation optimiert wird.

#### Kleinere Dimensionen

In [ ]:
using LinearAlgebra

BLAS.get_config()

In [ ]:
function smarter_avx_matmul!(A, B, C)
    # die Umordnung des Loops wird automatisch gemacht!
    @turbo for i ∈ 1:size(A,1), j ∈ 1:size(B,2)
        # so ist es sogar noch ein bisschen schneller, vermutlich weil dieses Statement besser parallelisiert werden kann
        C[i,j] = 0.0
        for k ∈ 1:size(A,2)
            C[i,j] += A[i,k] * B[k,j]
        end
    end
end

In [ ]:
m = 100; n = 100; k = 100
X = rand(m, k)
Y = rand(k, n)
Z = zeros(m, n)

@btime smarter_avx_matmul!(X, Y, Z)

In [ ]:
@btime Z = X * Y

Weiter Tipps findet man in der [Dokumentation](https://docs.julialang.org/en/v1/manual/performance-tips/).

### Kontextabhängige Optimierung

Eine Sache, die man nicht vergessen darf, ist: Code ist oftmals nur für eine konkrete, spezielle Anwendung optimal. Kommen wir beispielsweise zu unseren Zufallszahlen von vorhin zurück. Da ist die Effizienz zum Teil abhängig davon, ob wir *oft* aus derselben Verteilung samplen, weil die Initialisierung des Samplers aufwendig sein kann.

In [ ]:
# sample from categorical distribution
function categorical(probabilities)
    u = rand()
    sum = 0.0
    for i in eachindex(probabilities)
        sum += probabilities[i]
        if u <= sum
            return i
        end
    end
end
@btime categorical([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.9])

Unser naiver Code ist langsamer als der Sampler aus Distributions.jl:

In [ ]:
dist_often = Categorical([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.9])
@btime rand(dist_often)

Dieser hat aber eine ziemlich schlechte Performance, wenn wir ihn in jeder Iteration neu definieren müssen:

In [ ]:
@btime begin 
    dist_often = Categorical([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.9])
    rand(dist_often)
end